Imports

In [1]:
import pandas as pd
import numpy as np
from pyomo.environ import * 
from pyomo.opt import SolverFactory
import folium
import csv

Cargar archivos requerido por la instancia

In [ ]:
clients_df = pd.read_csv("Proyecto_Caso_Base/clients.csv")
depots_df = pd.read_csv("Proyecto_Caso_Base/depots.csv")
vehicles_df = pd.read_csv("Proyecto_Caso_Base/vehicles.csv")
